In [1]:
# Cheers again to Brunna Torino for their great analysis of real estate prices in Amsterdam 
# (https://towardsdatascience.com/ai-and-real-state-renting-in-amsterdam-part-1-5fce18238dbc)
# The ideas have been repurposed here to deal with the Washington, DC real estate market!

In [15]:
import numpy as np
import pandas as pd

apts = pd.read_csv('ApartmentList_Cleaned_NoOutliers.csv')
apts_ml = apts[['apt_rent', 'apt_footage', 'rooms', 'distance_wh', 'distance_cap']]

In [17]:
apts_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   apt_rent      1997 non-null   int64  
 1   apt_footage   1856 non-null   float64
 2   rooms         1997 non-null   int64  
 3   distance_wh   1997 non-null   int64  
 4   distance_cap  1997 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 78.1 KB


In [13]:
target= np.array(apts_ml['apt_rent'])
features = apts_ml.drop('apt_rent', axis = 1)
feature_list = list(features.columns)
features = np.array(features)

## RANDOM FOREST - KFOLD AND MODEL 


from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
    
kf = KFold(n_splits=10,random_state=42,shuffle=True)
accuracies = []
for train_index, test_index in kf.split(features):

    data_train   = features[train_index]
    target_train = target[train_index]

    data_test    = features[test_index]
    target_test  = target[test_index]

    rf = RandomForestRegressor(n_estimators = 1000, 
                               random_state = 42, 
                               criterion = 'mse',
                               bootstrap=True)
    
    rf.fit(data_train, target_train)

    predictions = rf.predict(data_test)

    errors = abs(predictions - target_test)

    print('Mean Absolute Error:', round(np.mean(errors), 2))
    
    mape = 100 * (errors / target_test)
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print('Average accuracy:', average_accuracy)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').